# Dominik Adamczyk
## Laboratorium 1 - rozwiązania 

#### Zaimplementuj w Pythonie algorytmy wyszukiwania wzorców

##### Naiwny

In [2]:
def naive_string_matching(text, pattern):
    results = []
    for s in range(len(text) - len(pattern) + 1):
        if text[s:s+len(pattern)] == pattern:
            results.append(s)
    return results

### Algorytm KMP

In [3]:
def prefix_function(pattern):
    pi = [0]
    k = 0
    for q in range(1, len(pattern)):
        while (k > 0 and pattern[k] != pattern[q]):
            k = pi[k-1]
        if pattern[k] == pattern[q]:
            k = k+1
        pi.append(k)
    return pi

def kmp_string_matching(text, pattern, pi):
    q = 0
    results = []
    for i in range(0, len(text)):
        while q > 0 and pattern[q] != text[i]:
            q = pi[q-1]
        if pattern[q] == text[i]:
            q = q + 1
        if q == len(pattern):
            results.append(i - q + 1)
            q = pi[q-1]
    return results

#### Automat skończony


In [92]:
def transition_table(pattern, alphabet):
    result = []
    for q in range(0, len(pattern) + 1):
        if q %100 == 0:
            print(q)
        result.append({})
        for a in set(alphabet):
            k = min(len(pattern) + 1, q + 2)
            while True:
                k = k - 1
                if (k == 0 or pattern[:k] == (pattern[:q] + a)[-k:]):
                    break
            result[q][a] = k
    return result

def fa_string_matching(text, delta):
    q = 0
    results = []
    length = len(delta) - 1
    for i in range(0, len(text)):
        q = delta[q][text[i]]
        if q == length:
            results.append(i - q + 1)
    return results

#### Zaimplementuj testy porównujące szybkość działania wyżej wymienionych algorytmów, z rozbiciem na czasu pre-processingu oraz czas wyszukiwania wzorca w tekście

In [5]:
from time import time
import pandas as pd

def test_naive(text, pattern):
    t = time()
    naive_string_matching(text, pattern)
    return 0, time() - t

def test_kmp(text, pattern):
    t = time()
    pf = prefix_function(pattern)
    preprocess_time = time() - t
    t = time()
    kmp_string_matching(text, pattern, pf)
    return preprocess_time, time() - t

def test_fa(text, pattern):
    t = time()
    tt = transition_table(pattern, text)
    preprocess_time = time() - t
    t = time()
    fa_string_matching(text, tt)
    return preprocess_time, time() - t

def test_all(text, pattern, n):
    df = pd.DataFrame({"Algorithm": [], "Preprocess_time" : [],
                    "Search_time" : [], "Full_time" : []})
    for i in range(n):
        naive_pre, naive_sear = test_naive(text, pattern)
        kmp_pre, kmp_sear = test_kmp(text, pattern)
        fa_pre, fa_sear = test_fa(text, pattern)
        new_rows = pd.DataFrame({"Algorithm": ["Naive", "KMP", "Finite_automata"],
                              "Preprocess_time" : [naive_pre, kmp_pre, fa_pre],
                              "Search_time" : [naive_sear, kmp_sear, fa_sear],
                              "Full_time" : [naive_pre+naive_sear, kmp_pre+kmp_sear, fa_pre+fa_sear]})
        df = pd.concat([df, new_rows], ignore_index=True)
    return df



In [76]:
import random
import string
def generate_random_string(l):
    return ''.join(random.choice(string.ascii_letters
                                 + string.digits) for _ in range(l))

Losowy test 1.

In [78]:
text_r1 = generate_random_string(10000000)
pattern_r1 = generate_random_string(10)

In [81]:
result_r1 = test_all(text_r1, pattern_r1, 10)

In [85]:
means = result_r1.groupby('Algorithm').agg({'Preprocess_time':['mean'],
                                     'Search_time':['mean'],
                                     'Full_time':['mean', 'min', 'max']})
print(means)


                Preprocess_time Search_time Full_time                    
                           mean        mean      mean       min       max
Algorithm                                                                
Finite_automata        1.660419    1.001444  2.661863  2.594374  2.767302
KMP                    0.000000    1.541157  1.541157  1.479107  1.712364
Naive                  0.000000    1.732907  1.732907  1.620245  1.904849


Losowy test 2.

In [86]:
text_r2 = generate_random_string(10000000)
pattern_r2 = generate_random_string(1000)

In [88]:
result_r2 = test_all(text_r2, pattern_r2, 1)

In [341]:
means = result_r2.groupby('Algorithm').agg({'Preprocess_time':['mean'],
                                     'Search_time':['mean'],
                                     'Full_time':['mean', 'min', 'max']})
print(result_r2)

         Algorithm  Preprocess_time  Search_time   Full_time
0            Naive         0.000000     2.722198    2.722198
1              KMP         0.000000     1.616200    1.616200
2  Finite_automata       171.172154     1.042874  172.215029


#### Znajdź wszystkie wystąpienia wzorców "pan" oraz "pani" w załączonym pliku, za pomocą każdego algorytmu. W raporcie zamieść liczbę dopasowań każdego ze wzorców osobno dla każdego algorytmu. Upewnij się, że każdy algorytm zwraca taką samą liczbę dopasowań

In [39]:
file = open("pan-tadeusz.txt", "r")
pan_tadeusz = file.read()
file.close()

In [73]:
pattern1 = "pan"
pattern2 = "pani"

pan_naive = naive_string_matching(pan_tadeusz, pattern1)
pan_kmp = kmp_string_matching(pan_tadeusz, pattern1, prefix_function(pattern1))
pan_fa = fa_string_matching(pan_tadeusz, transition_table(pattern1, pan_tadeusz))

pani_naive = naive_string_matching(pan_tadeusz, pattern2)
pani_kmp = kmp_string_matching(pan_tadeusz, pattern2, prefix_function(pattern2))
pani_fa = fa_string_matching(pan_tadeusz, transition_table(pattern2, pan_tadeusz))

print(f"""Searching word \"pan\":
Naive algorithm: {len(pan_naive)}
kmp algorithm: {len(pan_kmp)}
finite automata algorithm: {len(pan_fa)}
Same results : {pan_naive == pan_kmp == pan_fa}\n""")

print(f"""Searching word \"pani\":
Naive algorithm: {len(pani_naive)}
kmp algorithm: {len(pani_kmp)}
finite automata algorithm: {len(pani_fa)}
Same results : {pani_naive == pani_kmp == pani_fa}\n""")

Searching word "pan":
naive algorithm: 401
kmp algorithm: 401
finite automata algorithm: 401
Same results : True


Searching word "pani":
naive algorithm: 100
kmp algorithm: 100
finite automata algorithm: 100
Same results : True


#### Porównaj szybkość działania algorytmów dla problemu z p. 3, z uwzględnieniem czasu pre-processingu oraz czasu dopasowania. Pomiar czasu powinien być przeprowadzony co najmniej 5-krotnie i przedstawione w formie tabeli oraz wykresu, uwzględniającego czas minimalny, maksymalny oraz średni czas

In [75]:
df = test_all(pan_tadeusz, "pan", 100) # 100 measurements
means = df.groupby('Algorithm').agg({'Preprocess_time':['mean'],
                                     'Search_time':['mean'],
                                     'Full_time':['mean', 'min', 'max']})
print(means)



                Preprocess_time Search_time Full_time                    
                           mean        mean      mean       min       max
Algorithm                                                                
Finite_automata        0.028935    0.046510  0.075445  0.062689  0.095877
KMP                    0.000000    0.072909  0.072909  0.057506  0.137906
Naive                  0.000000    0.079825  0.079825  0.071613  0.107293


In [102]:
text_5x = "aab" * 100000
pattern_5x = "aab" * 10000

In [103]:
# result_r3 = test_all(text_5x, pattern_5x, 1)
print(test_kmp(text_5x,pattern_5x))
print(test_naive(text_5x, pattern_5x))
print(test_fa(text_5x, pattern_5x))

(0.0009975433349609375, 0.1351633071899414)
(0, 0.10802960395812988)
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500


In [ ]:
means = result_r3.groupby('Algorithm').agg({'Preprocess_time':['mean'],
                                     'Search_time':['mean'],
                                     'Full_time':['mean', 'min', 'max']})
print(means)
print(result_r3)

                Preprocess_time Search_time   Full_time                        
                           mean        mean        mean         min         max
Algorithm                                                                      
Finite_automata      117.375464    1.274849  118.650313  118.650313  118.650313
KMP                    0.000000    3.484569    3.484569    3.484569    3.484569
Naive                  0.000000    3.508453    3.508453    3.508453    3.508453
         Algorithm  Preprocess_time  Search_time   Full_time
0            Naive         0.000000     3.508453    3.508453
1              KMP         0.000000     3.484569    3.484569
2  Finite_automata       117.375464     1.274849  118.650313
